In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout, concatenate

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Dataset

In [3]:
dataset = pd.read_csv('../../data/TFT_energy.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,price,price_dayahead,gen_coal,gen_gas,load_actual,gen_lig,gen_oil,gen_oth_renew,pressure_Barcelona,pressure_Bilbao,...,wind_deg_Bilbao,clouds_all_Bilbao,gen_hyd_river,wind_deg_Seville,wind_speed_Barcelona,wind_speed_Valencia,wind_speed_Bilbao,gen_wind,wind_speed_Madrid,gen_hyd_pump
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,64.92,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,...,229.0,0.0,1009.0,21.0,7.0,1.0,0.0,5890.0,1.0,920.0
2015-01-01 01:00:00+00:00,64.48,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,...,224.0,0.0,973.0,27.0,7.0,0.0,1.0,5461.0,1.0,1164.0
2015-01-01 02:00:00+00:00,59.32,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,...,225.0,0.0,949.0,27.0,7.0,0.0,1.0,5238.0,1.0,1503.0
2015-01-01 03:00:00+00:00,56.04,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,...,221.0,0.0,953.0,27.0,7.0,0.0,1.0,4935.0,1.0,1826.0
2015-01-01 04:00:00+00:00,53.63,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,...,224.0,0.0,952.0,57.0,5.0,2.0,1.0,4618.0,0.0,2109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 18:00:00+00:00,77.02,68.85,2628.0,7634.0,30653.0,0.0,178.0,95.0,1027.0,1033.0,...,57.0,0.0,1135.0,30.0,1.0,2.0,0.0,3113.0,1.0,1.0
2018-12-31 19:00:00+00:00,76.16,68.40,2566.0,7241.0,29735.0,0.0,174.0,95.0,1027.0,1034.0,...,0.0,0.0,1172.0,30.0,3.0,1.0,1.0,3288.0,1.0,1.0
2018-12-31 20:00:00+00:00,74.30,66.88,2422.0,7025.0,28071.0,0.0,168.0,94.0,1028.0,1034.0,...,140.0,0.0,1148.0,50.0,4.0,3.0,1.0,3503.0,1.0,50.0


In [10]:
# # Plot Pearson correlation matrix
# import seaborn as sns

# correlations = data.corr(method='pearson')
# fig = plt.figure(figsize=(24, 24))
# sns.heatmap(correlations, annot=True, fmt='.2f')
# plt.title('Pearson Correlation Matrix')
# plt.show()

In [11]:
# data.tail(10)

### Dataset Information

In [12]:
print(list(data.columns))

['price', 'price_dayahead', 'gen_coal', 'gen_gas', 'load_actual', 'gen_lig', 'gen_oil', 'gen_oth_renew', 'pressure_Barcelona', 'pressure_Bilbao', 'gen_waste', 'gen_bio', 'temp_min_Valencia', 'pressure_Valencia', 'temp_min_Barcelona', 'humidity_Seville', 'wind_deg_Bilbao', 'clouds_all_Bilbao', 'gen_hyd_river', 'wind_deg_Seville', 'wind_speed_Barcelona', 'wind_speed_Valencia', 'wind_speed_Bilbao', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']


### Pre-processing

In [13]:
# specify the number of lag hours
n_hours = 24*3
K = 24
n_features = data.shape[1]

#### Stage

In [14]:
# Pre-processing
# Stage --> 8 stages
pms = dataset[['price']]
print("pms.shape:", pms.shape)

# stages_supervised = stage_series_to_supervised(stages, n_hours, K, 1)
pms_supervised = series_to_supervised(pms, n_hours, K)
pms_supervised.reset_index(drop=True, inplace=True)
print("pms_supervised.shape:", pms_supervised.shape)

pms.shape: (35063, 1)
pms_supervised.shape: (34968, 96)


In [15]:
# pms_supervised

#### Prior known covariates (9)

In [16]:
# non_stages = data[['WS_S4', 'FLOW_S25A', 'GATE_S25A', 'FLOW_S25B', 'GATE_S25B', 'FLOW_S26', 'GATE_S26', 'PUMP_S26', 'PUMP_S25B', 'MEAN_RAIN']]
prior_known_cov = data[['price_dayahead','gen_coal', 'gen_gas', 'load_actual', 'gen_lig', 'gen_oil', \
                        'gen_oth_renew', 'pressure_Barcelona', 'pressure_Bilbao', \
                        'gen_waste', 'gen_wind', 'wind_speed_Madrid', 'gen_hyd_pump']]
print("prior_known_cov.shape:", prior_known_cov.shape)

prior_known_cov_supervised = series_to_supervised(prior_known_cov, n_hours, K)
print("prior_known_cov_supervised.shape:", prior_known_cov_supervised.shape)

prior_known_cov.shape: (35063, 13)
prior_known_cov_supervised.shape: (34968, 1248)


In [17]:
prior_known_cov_supervised.reset_index(drop=True, inplace=True)
prior_known_cov_supervised.columns = ['past_known_cov_' + i for i in list(prior_known_cov_supervised.columns)]

In [18]:
prior_known_cov_supervised

,past_known_cov_var1(t-72),past_known_cov_var2(t-72),past_known_cov_var3(t-72),past_known_cov_var4(t-72),past_known_cov_var5(t-72),past_known_cov_var6(t-72),past_known_cov_var7(t-72),past_known_cov_var8(t-72),past_known_cov_var9(t-72),past_known_cov_var10(t-72),...,past_known_cov_var4(t+23),past_known_cov_var5(t+23),past_known_cov_var6(t+23),past_known_cov_var7(t+23),past_known_cov_var8(t+23),past_known_cov_var9(t+23),past_known_cov_var10(t+23),past_known_cov_var11(t+23),past_known_cov_var12(t+23),past_known_cov_var13(t+23)
0,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,195.0,...,25069.0,326.0,352.0,72.0,1030.0,1033.0,158.0,7351.0,1.0,650.0
1,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,196.0,...,23390.0,104.0,310.0,74.0,1027.0,1032.0,153.0,7403.0,1.0,938.0
2,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,191.0,...,21988.0,0.0,311.0,75.0,1027.0,1032.0,156.0,7887.0,1.0,1021.0
3,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,189.0,...,21182.0,0.0,308.0,71.0,1027.0,1031.0,155.0,8348.0,1.0,2067.0
4,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,188.0,...,20778.0,0.0,308.0,71.0,1026.0,1031.0,155.0,8348.0,1.0,2067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,70.00,2923.0,13280.0,33142.0,721.0,308.0,95.0,1027.0,1028.0,307.0,...,30653.0,0.0,178.0,95.0,1027.0,1033.0,277.0,3113.0,1.0,1.0
34964,70.01,2932.0,13091.0,32669.0,721.0,308.0,98.0,1027.0,1028.0,306.0,...,29735.0,0.0,174.0,95.0,1027.0,1034.0,280.0,3288.0,1.0,1.0
34965,68.71,2941.0,11128.0,30753.0,721.0,307.0,95.0,1027.0,1029.0,304.0,...,28071.0,0.0,168.0,94.0,1028.0,1034.0,286.0,3503.0,1.0,50.0
34966,68.40,2921.0,9240.0,27882.0,705.0,306.0,96.0,1027.0,1029.0,308.0,...,25801.0,0.0,163.0,93.0,1028.0,1034.0,287.0,3586.0,2.0,108.0


In [19]:
prior_known_future_cov = prior_known_cov_supervised.iloc[:, n_hours*prior_known_cov.shape[1]:]
prior_known_future_cov

,past_known_cov_var1(t),past_known_cov_var2(t),past_known_cov_var3(t),past_known_cov_var4(t),past_known_cov_var5(t),past_known_cov_var6(t),past_known_cov_var7(t),past_known_cov_var8(t),past_known_cov_var9(t),past_known_cov_var10(t),...,past_known_cov_var4(t+23),past_known_cov_var5(t+23),past_known_cov_var6(t+23),past_known_cov_var7(t+23),past_known_cov_var8(t+23),past_known_cov_var9(t+23),past_known_cov_var10(t+23),past_known_cov_var11(t+23),past_known_cov_var12(t+23),past_known_cov_var13(t+23)
0,20.20,2755.0,3997.0,23620.0,0.0,310.0,58.0,1027.0,1037.0,160.0,...,25069.0,326.0,352.0,72.0,1030.0,1033.0,158.0,7351.0,1.0,650.0
1,19.16,2678.0,3787.0,22606.0,0.0,306.0,57.0,1027.0,1037.0,161.0,...,23390.0,104.0,310.0,74.0,1027.0,1032.0,153.0,7403.0,1.0,938.0
2,21.90,2426.0,3724.0,22191.0,0.0,311.0,56.0,1028.0,1036.0,162.0,...,21988.0,0.0,311.0,75.0,1027.0,1032.0,156.0,7887.0,1.0,1021.0
3,14.90,2046.0,3578.0,21929.0,0.0,310.0,56.0,1027.0,1036.0,161.0,...,21182.0,0.0,308.0,71.0,1027.0,1031.0,155.0,8348.0,1.0,2067.0
4,24.65,2248.0,3676.0,22195.0,0.0,310.0,58.0,1027.0,1036.0,157.0,...,20778.0,0.0,308.0,71.0,1026.0,1031.0,155.0,8348.0,1.0,2067.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,67.83,1556.0,5683.0,30042.0,0.0,221.0,97.0,1027.0,1034.0,311.0,...,30653.0,0.0,178.0,95.0,1027.0,1033.0,277.0,3113.0,1.0,1.0
34964,68.40,1616.0,5859.0,30229.0,0.0,216.0,96.0,1027.0,1035.0,314.0,...,29735.0,0.0,174.0,95.0,1027.0,1034.0,280.0,3288.0,1.0,1.0
34965,67.64,1610.0,5660.0,29145.0,0.0,215.0,98.0,1028.0,1035.0,316.0,...,28071.0,0.0,168.0,94.0,1028.0,1034.0,286.0,3503.0,1.0,50.0
34966,65.27,1622.0,5603.0,26934.0,0.0,225.0,98.0,1028.0,1035.0,309.0,...,25801.0,0.0,163.0,93.0,1028.0,1034.0,287.0,3586.0,2.0,108.0


In [20]:
prior_known_future_cov.shape

(34968, 312)

### N_out, N_in, K

In [21]:
# specify the number of lag hours
n_features = pms.shape[1] + prior_known_cov.shape[1]
n_features

14

### Concatenation

In [22]:
past_prior_known_cov_supervised = prior_known_cov_supervised.iloc[:, :n_hours*prior_known_cov.shape[1]]

In [23]:
columns = []
for i in range(n_hours):
    #columns = columns + past_prior_unknown_cov_supervised.columns[i*prior_unknown_cov.shape[1]:(i+1)*prior_unknown_cov.shape[1]].tolist()
    columns = columns + past_prior_known_cov_supervised.columns[i*prior_known_cov.shape[1]:(i+1)*prior_known_cov.shape[1]].tolist()
# columns

In [24]:
past_unknown_known_cov = concat([
                                #past_prior_unknown_cov_supervised, 
                                past_prior_known_cov_supervised],
                                axis=1)

In [25]:
past_unknown_known_cov

,past_known_cov_var1(t-72),past_known_cov_var2(t-72),past_known_cov_var3(t-72),past_known_cov_var4(t-72),past_known_cov_var5(t-72),past_known_cov_var6(t-72),past_known_cov_var7(t-72),past_known_cov_var8(t-72),past_known_cov_var9(t-72),past_known_cov_var10(t-72),...,past_known_cov_var4(t-1),past_known_cov_var5(t-1),past_known_cov_var6(t-1),past_known_cov_var7(t-1),past_known_cov_var8(t-1),past_known_cov_var9(t-1),past_known_cov_var10(t-1),past_known_cov_var11(t-1),past_known_cov_var12(t-1),past_known_cov_var13(t-1)
0,48.10,4755.0,5196.0,24382.0,328.0,158.0,71.0,1035.0,1035.0,195.0,...,25254.0,0.0,323.0,58.0,1028.0,1037.0,162.0,8540.0,0.0,1045.0
1,47.33,4581.0,4857.0,22734.0,323.0,157.0,73.0,1036.0,1036.0,196.0,...,23620.0,0.0,310.0,58.0,1027.0,1037.0,160.0,8080.0,0.0,1257.0
2,42.27,4131.0,4314.0,21286.0,254.0,160.0,75.0,1036.0,1035.0,191.0,...,22606.0,0.0,306.0,57.0,1027.0,1037.0,161.0,7773.0,1.0,1629.0
3,38.41,3840.0,4130.0,20264.0,187.0,156.0,74.0,1036.0,1035.0,189.0,...,22191.0,0.0,311.0,56.0,1028.0,1036.0,162.0,7601.0,1.0,1962.0
4,35.72,3590.0,4038.0,19905.0,178.0,156.0,74.0,1037.0,1035.0,188.0,...,21929.0,0.0,310.0,56.0,1027.0,1036.0,161.0,7584.0,1.0,2185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34963,70.00,2923.0,13280.0,33142.0,721.0,308.0,95.0,1027.0,1028.0,307.0,...,29221.0,0.0,228.0,96.0,1027.0,1034.0,311.0,5129.0,1.0,1.0
34964,70.01,2932.0,13091.0,32669.0,721.0,308.0,98.0,1027.0,1028.0,306.0,...,30042.0,0.0,221.0,97.0,1027.0,1034.0,311.0,5406.0,1.0,1.0
34965,68.71,2941.0,11128.0,30753.0,721.0,307.0,95.0,1027.0,1029.0,304.0,...,30229.0,0.0,216.0,96.0,1027.0,1035.0,314.0,5193.0,0.0,1.0
34966,68.40,2921.0,9240.0,27882.0,705.0,306.0,96.0,1027.0,1029.0,308.0,...,29145.0,0.0,215.0,98.0,1028.0,1035.0,316.0,4912.0,1.0,1.0


In [26]:
# non_stages_supervised.reset_index(drop=True, inplace=True)
# stages_supervised.reset_index(drop=True, inplace=True)

all_data = concat([past_unknown_known_cov, pms_supervised.iloc[:, :]], axis=1)

In [27]:
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (34968, 1032)


In [29]:
# all_data

### Train & Test set

In [30]:
all_data = all_data.values
future_covariates = prior_known_future_cov.values

In [31]:
n_train_hours = int(len(data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]    # 0 column is the rainfall to measure heavy/medium/light
test = all_data[n_train_hours:, :]

n_train_hours: 28050


In [32]:
train_future_cov = prior_known_future_cov.iloc[:n_train_hours, :]
test_future_cov = prior_known_future_cov.iloc[n_train_hours:, :] 

In [33]:
test_future_cov.shape

(6918, 312)

### Normalization

In [34]:
n_hours

72

In [35]:
n_features

14

In [36]:
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (28050, 1008) (28050, 24) (6918, 1008) (6918, 24)


In [37]:
# normalize features
scaler1 = MinMaxScaler(feature_range=(0, 1))

train_future_cov = scaler1.fit_transform(train_future_cov)
test_future_cov = scaler1.fit_transform(test_future_cov)
print("train_future_cov.shape, test_future_cov.shape:", train_future_cov.shape, test_future_cov.shape)

train_future_cov.shape, test_future_cov.shape: (28050, 312) (6918, 312)


In [38]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (28050, 72, 14) (28050, 24) (6918, 72, 14) (6918, 24)


In [32]:
train_future_cov = train_future_cov.reshape(train_future_cov.shape[0], K, -1)
test_future_cov = test_future_cov.reshape(test_future_cov.shape[0], K, -1)
print("train_future_cov.shape, test_future_cov.shape:", train_future_cov.shape, test_future_cov.shape)

train_future_cov.shape, test_future_cov.shape: (28050, 24, 13) (6918, 24, 13)


In [50]:
train_y = train_y.reshape((train_y.shape[0], K, -1))
test_y = test_y.reshape((test_y.shape[0], K, -1))
print(train_y.shape, test_y.shape)

(28050, 24, 1) (6918, 24, 1)


In [40]:
# past_inputs

### Model

In [45]:
# latent_dim = 128

# Encoding the past
past_inputs = tf.keras.Input(shape=(n_hours, n_features), name='past_inputs')

# encoder = tf.keras.layers.SimpleRNN(96, activation='relu', return_state=True, return_sequences=False)
encoder = tf.keras.layers.SimpleRNN(96, activation='relu', return_state=True, return_sequences=True)
encoder_outputs, state_h = encoder(past_inputs)  # state_h, state_c for LSTM


# Adding the future
future_inputs = tf.keras.Input(shape=(K, prior_known_cov.shape[1]), name='future_inputs')
decoder_lstm = tf.keras.layers.SimpleRNN(96, activation='relu', return_sequences=True)


# Past + Future
x = decoder_lstm(future_inputs, initial_state=[state_h])   # state_h, state_c


# Output
x = tf.keras.layers.Dense(256, activation='relu')(x)

output = tf.keras.layers.Dense(pms.shape[1])(x)

# Combining future inputs with recurrent branch output
model = tf.keras.models.Model(inputs=[past_inputs, future_inputs], outputs=output)
model.summary()

In [49]:
lr = 0.00001
EPOCHS = 1000
model.compile(
              optimizer=Adam(learning_rate=lr, decay=lr/EPOCHS),
#               optimizer='adam',
              loss='mse',
              metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=300)
mc = ModelCheckpoint('saved_model/rnn_encoder_decoder_24h_energy.h5', monitor='val_mae', mode='min', verbose=2, save_best_only=True)

history = model.fit(x=[train_X, train_future_cov], y=train_y, 
                    epochs=EPOCHS,
                    verbose=2,
                    validation_data=([test_X, test_future_cov], test_y),
                    batch_size = 512,
                    callbacks=[es, mc]
                   )

plt.rcParams["figure.figsize"] = (8, 6)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vds Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/1000


ValueError: in user code:

    /aul/homes/jshi008/miniconda3/envs/tf22/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /aul/homes/jshi008/miniconda3/envs/tf22/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /aul/homes/jshi008/miniconda3/envs/tf22/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /aul/homes/jshi008/miniconda3/envs/tf22/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /aul/homes/jshi008/miniconda3/envs/tf22/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /aul/hom

In [38]:
from tensorflow.keras.models import load_model

model = load_model('saved_model/rnn_encoder_decoder_24h_energy.h5')

### Prediction

In [39]:
y_hat = model.predict([test_X, test_future_cov])

In [40]:
print(y_hat.shape)
y_hat = y_hat.reshape((-1, y_hat.shape[1]*y_hat.shape[2]))
print(y_hat.shape)

(6918, 24, 1)
(6918, 24)


In [41]:
print(test_y.shape)
test_y = test_y.reshape((-1, test_y.shape[1]*test_y.shape[2]))
# print(test_y.shape)

(6918, 24, 1)


In [42]:
inv_yhat = scaler.inverse_transform(y_hat)
inv_y = scaler.inverse_transform(test_y)

inv_yhat = pd.DataFrame(inv_yhat)
inv_y = pd.DataFrame(inv_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)

inv_y.shape, inv_yhat.shape (6918, 24) (6918, 24)


### Performance

In [43]:
# K=24h
print('MAE = {}'.format(float("{:.3f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
print('RMSE = {}'.format(float("{:.3f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

MAE = 3.895
RMSE = 5.318
